In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
client_id = 'dbdbbe596bcc4986baff23591208c5d3'
client_secret = '6461245bf7524960b030e5d8e2228df7'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# Retrieve data for a set of tracks 
playlist_id = '37i9dQZEVXcLTG8F9rbmbd'
playlist_tracks = sp.playlist_tracks(playlist_id)

# Extract audio features for each track in the playlist
playlist_track_features = []
playlist_track_names = []
for track in playlist_tracks['items']:
    track_id = track['track']['id']
    features = sp.audio_features([track_id])[0]
    track_name = track['track']['name']
    playlist_track_names.append(track_name)
    playlist_track_features.append([features['danceability'], features['energy'], features['valence'], features['acousticness'], features['instrumentalness'], features['liveness']])

# Convert playlist track features to numpy array
playlist_track_features_array = np.array(playlist_track_features)

# Define new songs (not in the playlist)
new_song_ids = ['0djZ2ndRfAL69WYNra5jRC', '6UFivO2zqqPFPoQYsEMuCc', '4c2ZCkr6k0xsfHp7VfWzhg']
new_song_titles = []
new_song_features = []

# Get song titles and extract audio features for new songs
for song_id in new_song_ids:
    track_info = sp.track(song_id)
    new_song_titles.append(track_info['name'])
    features = sp.audio_features([song_id])[0]
    new_song_features.append([features['danceability'], features['energy'], features['valence'], features['acousticness'], features['instrumentalness'], features['liveness']])

# Convert new song features to numpy array
new_song_features_array = np.array(new_song_features)

# Compute pairwise distances between playlist and new songs
pairwise_distances_new_songs = pairwise_distances(playlist_track_features_array, new_song_features_array, metric='euclidean')

# Convert distances to similarity scores
max_distance = pairwise_distances_new_songs.max()
cosine_sim_new_songs_sklearn = 1 - (pairwise_distances_new_songs / max_distance)

# Prepare column names for the dataframe
column_names = [f"{title} by {sp.track(song_id)['artists'][0]['name']}" for title, song_id in zip(new_song_titles, new_song_ids)]

# Create a DataFrame to store similarity scores between playlist tracks and new songs
similarity_new_songs_df_sklearn = pd.DataFrame(cosine_sim_new_songs_sklearn, index=playlist_track_names, columns=column_names)

# Output the DataFrame
print("Similarity Scores between Playlist Tracks and New Songs using SKLearn:")
similarity_new_songs_df_sklearn

Similarity Scores between Playlist Tracks and New Songs using SKLearn:


,April Come She Will by Simon & Garfunkel,Bags by Clairo,Didn't Want To Have To Do It (Ver. 1) by Cass Elliot
Roommates,0.671138,0.359297,0.718254
鉄鎖,0.388697,0.152609,0.509444
Turn Into,0.047091,0.608055,0.178013
The Moon,0.071971,0.151734,0.199449
When You're Smiling and Astride Me,0.355076,0.564700,0.499790
Midnight,0.193102,0.641602,0.325041
Ragged Wood,0.220978,0.252083,0.397417
The Flower Called Nowhere,0.561469,0.384859,0.669054
Beware of the Dogs,0.309748,0.449922,0.463174
Stella Brown,0.384224,0.595508,0.492873


In [9]:
# Initialize an empty list to store the top similar pairings
top_similar_pairings = []

# Iterate over each playlist track
for playlist_track_name in playlist_track_names:
    # Get the similarity scores for the current playlist track
    similarity_scores = similarity_new_songs_df_sklearn.loc[playlist_track_name]
    
    # Sort the similarity scores in descending order
    sorted_indices = similarity_scores.argsort()[::-1]
    
    # Select the top 10 indices
    top_10_indices = sorted_indices[:10]
    
    # Create a list of tuples containing the similar pairing and their similarity scores
    similar_pairings = [(playlist_track_name, similarity_scores.index[i], similarity_scores[i]) for i in top_10_indices]
    
    # Append the list of similar pairings to the top_similar_pairings list
    top_similar_pairings.extend(similar_pairings)

# Sort the top similar pairings based on similarity scores in chronological order
top_similar_pairings.sort(key=lambda x: x[2], reverse=True)

# Output the list of top 10 similar pairings
print("Top 10 Similar Pairings:")
for i, pairing in enumerate(top_similar_pairings[:10], start=1):
    print(f"{i}. {pairing[0]} - {pairing[1]} (Similarity Score: {pairing[2]:.4f})")


Top 10 Similar Pairings:
1. We're Going to Be Friends - Didn't Want To Have To Do It (Ver. 1) by Cass Elliot (Similarity Score: 0.8359)
2. Life Will Be - Didn't Want To Have To Do It (Ver. 1) by Cass Elliot (Similarity Score: 0.8358)
3. I Wait for You - April Come She Will by Simon & Garfunkel (Similarity Score: 0.8300)
4. This Feeling - Didn't Want To Have To Do It (Ver. 1) by Cass Elliot (Similarity Score: 0.8140)
5. This Feeling - April Come She Will by Simon & Garfunkel (Similarity Score: 0.8138)
6. I Wait for You - Didn't Want To Have To Do It (Ver. 1) by Cass Elliot (Similarity Score: 0.7605)
7. Roommates - Didn't Want To Have To Do It (Ver. 1) by Cass Elliot (Similarity Score: 0.7183)
8. We're Going to Be Friends - April Come She Will by Simon & Garfunkel (Similarity Score: 0.7035)
9. j's lullaby (darlin' i'd wait for you) - Didn't Want To Have To Do It (Ver. 1) by Cass Elliot (Similarity Score: 0.6972)
10. Life Will Be - April Come She Will by Simon & Garfunkel (Similarity Scor

In [14]:
# Titles of the three specific songs
specific_songs = [
    "April Come She Will by Simon & Garfunkel",
    "Didn't Want To Have To Do It (Ver. 1) by Cass Elliot",
    "Bags by Clairo"
]

# Initialize a dictionary to store the top similar pairings for each specific song
top_similar_pairings_specific = {song: [] for song in specific_songs}

# Iterate over each of the three specific songs
for specific_song in specific_songs:
    # Get the similarity scores for the current specific song
    similarity_scores = similarity_new_songs_df_sklearn[specific_song]
    
    # Sort the similarity scores in descending order and get the top 10 indices
    top_10_indices = similarity_scores.argsort()[::-1][:10]
    
    # Retrieve the corresponding playlist track names and their scores
    top_similar_pairings = [(playlist_track_names[idx], similarity_scores[idx]) for idx in top_10_indices]
    
    # Store the top similar pairings in the dictionary
    top_similar_pairings_specific[specific_song] = top_similar_pairings

# Print the top 10 similar pairings for each of the three songs along with their average similarity scores
for song, pairings in top_similar_pairings_specific.items():
    print(f"Top 10 Similar Pairings for '{song}':")
    total_score = 0
    for i, (track_name, score) in enumerate(pairings, start=1):
        print(f"{i}. {track_name} (Similarity Score: {score:.4f})")
        total_score += score
    average_score = total_score / len(pairings)
    print(f"Average Similarity Score: {average_score:.4f}\n")  # Print the average and add a space for readability

Top 10 Similar Pairings for 'April Come She Will by Simon & Garfunkel':
1. I Wait for You (Similarity Score: 0.8300)
2. This Feeling (Similarity Score: 0.8138)
3. We're Going to Be Friends (Similarity Score: 0.7035)
4. Life Will Be (Similarity Score: 0.6827)
5. Roommates (Similarity Score: 0.6711)
6. j's lullaby (darlin' i'd wait for you) (Similarity Score: 0.6156)
7. I Kissed a Girl, And She Kissed Me (Similarity Score: 0.6124)
8. The Flower Called Nowhere (Similarity Score: 0.5615)
9. Forever (Similarity Score: 0.4966)
10. 鉄鎖 (Similarity Score: 0.3887)
Average Similarity Score: 0.6376

Top 10 Similar Pairings for 'Didn't Want To Have To Do It (Ver. 1) by Cass Elliot':
1. We're Going to Be Friends (Similarity Score: 0.8359)
2. Life Will Be (Similarity Score: 0.8358)
3. This Feeling (Similarity Score: 0.8140)
4. I Wait for You (Similarity Score: 0.7605)
5. Roommates (Similarity Score: 0.7183)
6. j's lullaby (darlin' i'd wait for you) (Similarity Score: 0.6972)
7. The Flower Called Nowh